In [1]:
import re
import pickle
import pandas as pd

# Natural Language Toolkit (nltk)
from nltk.stem import WordNetLemmatizer
from nltk import download
download('stopwords')
download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\DANG XUAN
[nltk_data]     BACH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\DANG XUAN
[nltk_data]     BACH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

In [3]:
mystopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [5]:
stopwordlist = stopwords.words('english') + mystopwordlist

In [6]:
def preprocess(textdata):
    processedText = []
    #creating a Lemmatizer
    wordLemma = WordNetLemmatizer() #define the imported library
    # Defining regular expression pattern we can find. in tweets
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)" 
    userPattern       = '@[^\s]+' # e.g @FagbamigbeK check this out
    alphaPattern      = "[^a-zA-Z0-9]" # e.g I am *10 better!
    sequencePattern   = r"(.)\1\1+"  # e.g Heyyyyyyy, I am back!
    seqReplacePattern = r"\1\1" # e.g Replace Heyyyyyyy with Heyy
    
    for tweet in textdata:
        tweet = tweet.lower() #normalizing all text to a lower case
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' URL',tweet) 
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])  
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' USER', tweet)  
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet) # e.g I am *10 better!
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet) # e.g Replace Heyyyyyyy with Heyy
         
        tweetwords = ''
        for word in tweet.split():
            if len(word) > 2 and word.isalpha():
                word = wordLemma.lemmatize(word)
                tweetwords += (word + ' ')
        processedText.append(tweetwords)
    return processedText

In [7]:
# def load_models():
#     # Load the vectoriser.
#     file = open('./vectoriser-ngram-(1,2).pickle', 'rb')
#     vectoriser = pickle.load(file)
#     file.close()
    
#     # Load the LGBM Model.
#     file = open('./Sentiment-LightGBM.pickle', 'rb')
#     LGBMmodel = pickle.load(file)
#     file.close()
    
#     return vectoriser, LGBMmodel

def predict(vectoriser, model, text):
    # Predict the sentiment
    textdata = vectoriser.transform(preprocess(text)) 
    sentiment = model.predict(textdata)
    
    # Make a list of text with sentiment.
    data = []
    for text, pred in zip(text, sentiment):
        data.append((text,pred))
        
    # Convert the list into a Pandas DataFrame.
    df = pd.DataFrame(data, columns = ['text','sentiment'])
    df = df.replace([0,1], ["Negative","Positive"])
    return df

In [8]:
#Để load lại:
import joblib
vectoriser = joblib.load('tfidf_vectoriser.pkl')

C:\Users\DANG XUAN BACH\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\DANG XUAN BACH\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
# Mở tệp chứa mô hình đã lưu
with open('Sentiment-LightGBM.pickle', 'rb') as file:
    LGBMmodel = pickle.load(file)

C:\Users\DANG XUAN BACH\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
if __name__=="__main__": 
    # Text to classify should be in a list.
    text = ["Today is so great!",
            "May the Good Lord be with you.", "I hate peanuts!",
            "Mr. Kehinde, what are you doing next? this is great!"]
    
    df = predict(vectoriser, LGBMmodel, text)
    print(df.head())

                                                text sentiment
0                                 Today is so great!  Positive
1                     May the Good Lord be with you.  Positive
2                                    I hate peanuts!  Negative
3  Mr. Kehinde, what are you doing next? this is ...  Positive


In [11]:
# import pandas as pd

# # Đọc dữ liệu từ file CSV vào DataFrame
# df = pd.read_csv("C:\\Users\\Admin\\Documents\\TLCN\\data\\hashtag_joebiden.csv")

# # Xác định cột chứa các câu bạn muốn kiểm tra
# text_column = 'tweet'
# df[text_column] = df[text_column].fillna('')

# # Lấy dữ liệu từ cột chứa các câu
# text_to_predict = df[text_column].tolist()

# # Thực hiện dự đoán bằng cách sử dụng hàm predict đã được định nghĩa trước đó
# result_df = predict(vectoriser, LGBMmodel, text_to_predict)

# # In kết quả
# print(result_df.head())

# # Lưu kết quả vào một tệp CSV nếu cần
# result_df.to_csv('predicted_results.csv', index=False)


In [ ]:
{"created_at":"2024-03-14 20:30:19", "tweet_id": "1768268422909939980", "tweet": "Guardrails vs. Microsoft Guidance: A Hands-On Tutorial for LLM Output Validation by Daniel Klitzke at #ITNEXT. #python #llm #mlops #machinelearning #datascience https://t.co/z7htmzTEQF"}

In [ ]:
spark-submit --master local[*] --packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.4.0,mysql:mysql-connector-java:8.0.33 ab.py